# Постановка задачи



###  *Cогласно легенде задачи*
*Мы работаем в компании, которая занимается продажей автомобилей с пробегом в Москве.*  
*Основная задача компании и её менеджеров — максимально быстро находить выгодные предложения (проще говоря, купить ниже рынка, а продать дороже рынка).*  
   
<img src="https://universegadgets.com/userfiles/165/381_0.webp"/>   
   
*Руководство компании просит нашу команду создать **модель, которая будет предсказывать стоимость автомобиля по его характеристикам**.*  
*Только вот незадача: исторически сложилось, что компания изначально не собирала данные. Есть только небольшой датасет с историей продаж за короткий период, которого для обучения модели будет явно мало. Его мы будем использовать для теста, остальное придется собрать самим.*  

Согласно условиям задачи завершающим этапом является участие в [сореновании на Kaggle](https://www.kaggle.com/c/sf-dst-car-price-prediction). Метрика используемая в соревновании - MAPE  (Mean Percentage Absolute Error) 

Задача естетсвенным образом разбивается на три этапа:
- **Этап I:Сбор и подгтовка данных:** 
- **Этап II: EDA полученных данных**
- **Этап III: Создание, настройка модели, участие в соревновании**
   
   
### <a name="_"></a>План работы: 
- [Импорт библиотек, установка параметров, определение функций](#0)
 - [Импорт библиотек](#0_0)
 - [Установка параметров](#0_1)
 - [Определение функций](#0_2)
- [Ознакоомление -- Анализ валидационного набора данных](#1):
 - [Общий обзор](#1_0)
 - [Отбор принимаемых к расмотрению моделью данных](#1_1)
- [Сбор и подгтовка данных](#2):
  - [парсинг данных (https://auto.ru/moskva/)](#2_0)
  - [привлечение сторонних данных](#2_1)
  - [предобработка полученых данных (как парсинг, так и стороннние), приведение их к единому виду](#2_2)
   - [дубли и пропуски](#2_2_0)
   - [приведение к единому виду](#2_2_1)
- [EDA полученных данных](#3)
  - [EDA собсвенного набора](#3_0)
  - [EDA сторонних данных](#3_1)
  - [Сравнение распределений в наборах данных](#3_2)
  - [Выводы, предварительный выбор модели, варианты обработки данных](#3_3)
- [Создание, настройка модели, участие в соревновании](#4)
  - создание Baseline, эксперименты с Baseline
      - варианты обработки признаков
      - features engineering
      - сокращение признкового пространсва
  - выбор моделей 
      - эксперименты с моделлями
      - создание признаков
      - отбор признаков
      - подбор гиперпараметров модели
  - анализ смещения цен, поиск поправочных коэффициентов
      - анализ смещения
      - нахождение корректирующих коэффициентов
- [Ансамблирование](#5)
   - Стекинг
   - Блендинг

# <a name="0"></a> Импорт библиотек, установка параметров, определение функций
## <a name="0_0"></a> Импорт бибилиотек
вернуться к [Плану работы](#_)

In [1]:
import pandas as pd
import numpy as np
import json
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import date

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm
from bs4 import BeautifulSoup

import warnings

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## <a name="0_1"></a> Установка параметров и определение констант
вернуться к [Плану работы](#_)

In [2]:
from project_7_constants import *

In [3]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

## <a name="0_2"></a> Определение функций
вернуться к [Плану работы](#_)

In [4]:
from project_7_functions import *

# <a name="1"></a> Анализ  валидационных данных 

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо узнать какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем набор данных эти наблюдения описывающие.

In [5]:
df_test = pd.read_csv(f'{path}test.csv')

## <a name="1_0"></a> EDA валидационного набора
### Общий обзор
вернуться к [Плану работы](#_)

In [6]:
df_test.info(verbose=False)
describe_df(df_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


,count,unique,top,freq,min,max,type,NaN_prop
bodyType,34686,24,седан,13147,NaN,NaN,<class 'str'>,0.000
brand,34686,12,BMW,4473,NaN,NaN,<class 'str'>,0.000
car_url,34686,34686,https://auto.ru/cars/used/sale,1,NaN,NaN,<class 'str'>,0.000
color,34686,16,чёрный,11010,NaN,NaN,<class 'str'>,0.000
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegan",51,NaN,NaN,<class 'float'>,0.815
description,34686,31732,Выгода до 82 000 руб. при обме,264,NaN,NaN,<class 'str'>,0.000
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,<class 'str'>,0.000
enginePower,34686,315,249 N12,1708,NaN,NaN,<class 'str'>,0.000
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,<class 'str'>,0.288
fuelType,34686,5,бензин,28601,NaN,NaN,<class 'str'>,0.000


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков:   
-бинарные (в колонках `vendor`, `ПТС`, `Руль`);   
-категориальные (например в колонках `bodyType`, `brand`, `color`, `Владельцы`);  
-количественные (например в `enginePower` или `mileage`).  

Некоторын колонки содержат словари или длинные строки, содержащие сразу несколько признаков. 

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по этим 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на обьявления о продаже авто на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru***. Воспользуемся этим же источником.

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключим их из рассмотрения.  
  
Теперь следует ознакомиться с  каждым признак/колонку более детально. Это позволит нам понять в каком виде/формате содержится информация в колонках и соответсвенно сформировать набор данных для обучения в том же виде/формате. Так же детальное ознакомлене позволит окончательно определится с необходимость включения информации из колонок в рассмотрение моделью. В случае включения данных в рассмотрение моделью, проведем EDA.

## <a name="1_1"></a> Детальное ознакомление 
вернуться к [Плану работы](#_)   

 тут текст о выборе признков     
   
   
  
### bodyType

In [7]:
df_test.bodyType.unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

Колонка содержит наименование типа кузова в виде строковой величины. Принимается в пассмоттение.
### brand

In [8]:
df_test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

Колонка содержит наименование марки в виде строковой величины. Принимается в рассмотрение.

### car_url

In [9]:
df_test.car_url.sample(3)

1330     https://auto.ru/cars/used/sale/skoda/octavia/1...
34267    https://auto.ru/cars/used/sale/honda/civic/109...
16635    https://auto.ru/cars/used/sale/mercedes/glc_co...
Name: car_url, dtype: object

В этой колонке содержаться ссылки на обьявления о продаже, послужившие источником при парсинге `test`. В настоящий момент скорее всего ссылки уже не активны, потому колокна будет исключена из рассмотрения моделью. Но нформация в ней содержащаяся позволит нам проверить не попадают ли в тренировочный набор данных наблюдения из валидационного. 
### color

In [10]:
df_test.color.unique()

array(['синий', 'чёрный', 'серый', 'коричневый', 'белый', 'пурпурный',
       'бежевый', 'серебристый', 'красный', 'зелёный', 'жёлтый',
       'голубой', 'оранжевый', 'фиолетовый', 'золотистый', 'розовый'],
      dtype=object)

Колонка содержит название цвета кузова, принимется в рассмотрение.
### complectation_dict

In [11]:
df_test.complectation_dict.head(3)

0                                                  NaN
1                                                  NaN
2    {"id":"20026336","name":"Ambition","available_...
Name: complectation_dict, dtype: object

In [12]:
df_test.complectation_dict.loc[2]

'{"id":"20026336","name":"Ambition","available_options":["heated-wash-system","airbag-passenger","lock","door-sill-panel","electro-mirrors","mirrors-heat","cooling-box","computer","seat-transformation","wheel-power","fabric-seats","airbag-side","abs","wheel-leather","climate-control-1","esp","adaptive-light","audiopreparation","ashtray-and-cigarette-lighter","front-centre-armrest","electro-window-back","16-inch-wheels","body-mouldings","condition","airbag-driver","isofix","aux","electro-window-front","light-sensor","hcc","ptf","rain-sensor","tyre-pressure","audiosystem-cd","front-seats-heat","wheel-configuration2","wheel-configuration1","immo","12v-socket","third-rear-headrest"]}'

Колонка содержит словарь стандартной комплектации с наименованием комплектации и списком обрудования (зачастую стандартного, имеющегося в любом автомобиле, например "12v-socket" или "lock") входящим в такую комплектацию. Доля пррпусков (свыше 0.8), перечисление оборудования присутсвуюшего в любом авто данной марки и модели и наличие колонки equipment_dict с гораздо меньшим количеством пропусков позволяет исключить колонку из рассмотрения.
### description

In [13]:
print(df_test.description[0])

Все автомобили, представленные в продаже, проходят тщательную проверку по более 40 параметрам. Предоставляем гарантию юридической чистоты, а так же год технической гарантии на двигатель и КПП. Бесплатный тест-драйв. Возможно оформление автомобиля в кредит!

Преимущества автокредитования:
— Первоначальный взнос от 0%;
— Более 30 кредитных программ;
— Процентная ставка от 6% годовых;
— Срок кредита от 6 месяцев до 7 лет;
— Оформление кредита по двум документам;
— Досрочное погашение без штрафов и комиссий;
— Сумма кредита до 2 млн рублей;
— Оформление КАСКО – по желанию;
— Без справок и поручителей.

Сотрудничаем с 12 аккредитованными и сертифицированными банками РФ, среднее время ожидания решения банка 20–30 минут.

При покупке автомобиля по программе «Trade-in – выгодный обмен» или в кредит, получите дополнительную скидку до 80 000 рублей на данный автомобиль!

Записаться на тест-драйв, а так же получить подробную информацию можно у специалистов автоцентра по указанному номеру телефона

В колонке содержится текст, относящийся скорее к продавцу, чем к конкретному автомобилю. Поэтому на данном этапе ( и вероятно вообще) колонка исключается из рассмотрения.
### engineDisplacement

In [14]:
df_test.engineDisplacement.unique()

array(['1.2 LTR', '1.6 LTR', '1.8 LTR', '2.0 LTR', '1.4 LTR', '1.3 LTR',
       '1.0 LTR', '3.6 LTR', '1.5 LTR', '1.9 LTR', '2.8 LTR', '1.1 LTR',
       '2.5 LTR', '4.2 LTR', '3.0 LTR', '4.0 LTR', '5.9 LTR', '2.7 LTR',
       '3.1 LTR', '2.4 LTR', '5.2 LTR', '3.2 LTR', '4.1 LTR', '6.3 LTR',
       '2.3 LTR', '6.0 LTR', '2.2 LTR', '3.7 LTR', '2.9 LTR', '5.0 LTR',
       '3.3 LTR', '2.1 LTR', '2.6 LTR', ' LTR', '3.5 LTR', '1.7 LTR',
       '0.7 LTR', '4.4 LTR', '4.8 LTR', '5.4 LTR', '6.6 LTR', '4.9 LTR',
       '3.8 LTR', '3.4 LTR', '3.9 LTR', '4.6 LTR', '5.6 LTR', '4.5 LTR',
       '5.5 LTR', '6.2 LTR', '4.7 LTR', '4.3 LTR', '5.8 LTR', '5.3 LTR',
       '5.7 LTR'], dtype=object)

In [15]:
df_test.engineDisplacement.value_counts()[' LTR']

55

Колонка содержит обьем двигателя, принимется в рассмотрение. Следует отметить наличие 'скрытых' пропусков (55 стрк ' LTR')
### enginePower

In [16]:
df_test.enginePower.unique()[:20]

array(['105 N12', '110 N12', '152 N12', '200 N12', '102 N12', '150 N12',
       '90 N12', '180 N12', '220 N12', '122 N12', '70 N12', '140 N12',
       '125 N12', '54 N12', '86 N12', '75 N12', '64 N12', '95 N12',
       '260 N12', '170 N12'], dtype=object)

Колонка содержит мощность двигателя, принмается в рассмотрение.
### equipment_dict

In [17]:
df_test.equipment_dict.apply(lambda x: json.loads(x) if x==x else x).sample(5)

24679    {'airbag-driver': True, 'aux': True, 'electro-...
6580     {'cruise-control': True, 'alloy-wheel-disks': ...
27246    {'electro-window-back': True, 'alloy-wheel-dis...
20817                                                  NaN
7097     {'cruise-control': True, 'esp': True, 'multi-w...
Name: equipment_dict, dtype: object

В колонке много пропусков (почти 29%), не пропущенные значения представляют собой текстовое представление словаря с перечислением оснащения автомобиля.  
Самым простым способом обработки этой колонки является подсчет количества опций для каждого авто.  
Более трудоемким подходом является принятие к рассмтотрению моделью признаков-опций входящих в пересенчение множеств опций из обучающего и валидационного набора данных.  
Пока ограничемся первым вариантом.
### fuelType

In [18]:
df_test.fuelType.unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

Колонка содержит тип топлива. Принимается врассмотрнеи.

### image

In [19]:
df_test.image.sample(5)

26280    https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...
19091    https://autoru.naydex.net/LxPU17803/f5b8d77S/u...
11778    https://avatars.mds.yandex.net/get-autoru-vos/...
4187     https://autoru.naydex.net/ys1kR7800/fbd964zfcB...
2861     https://autoru.naydex.net/ys1kR7800/fbd964zfcB...
Name: image, dtype: object

Колонка содержит ссылки на фото или аватар авто, исключается из рассмотрения.
### mileage

In [20]:
df_test.mileage.sample(5)

29530    171500
25759    195739
23105    211000
25744    264000
20185    276415
Name: mileage, dtype: int64

Колонка содержит пробег авто, принмается в рассмотрениею

### modelDate

In [21]:
df_test.modelDate.unique()

array([2013, 2017, 2008, 2009, 2016, 2012, 2015, 2010, 2006, 2000, 2007,
       1994, 2004, 1999, 2005, 1976, 2001, 1969, 1996, 1998, 1989, 1934,
       2014, 2011, 2018, 1986, 1997, 1990, 2019, 2002, 1991, 1987, 1980,
       1982, 1938, 1988, 2003, 1983, 1978, 1979, 1984, 1992, 1995, 1993,
       1985, 1974, 1966, 1977, 1981, 1972, 1968, 1975, 1949, 1937, 1936,
       1973, 1959, 1958, 2020, 1965, 1971, 1904, 1963, 1955, 1951, 1960],
      dtype=int64)

Колонка содержит год начала выпуска модели, принимется в рассмотрние.

### model_info

In [22]:
df_test.model_info.sample(5)

29046    {"code":"LANCER","name":"Lancer","ru_name":"Ла...
32652    {"code":"JUKE","name":"Juke","ru_name":"Джук",...
6142     {"code":"CIVIC","name":"Civic","ru_name":"Циви...
2482     {"code":"A5","name":"A5","ru_name":"А5","morph...
33368    {"code":"RAV_4","name":"RAV4","ru_name":"Рав4"...
Name: model_info, dtype: object

Колонка содержит наименование модели в трех вариантах. Учитывая наличие model_name колонка исключается из рассмотрения
### model_name

In [23]:
df_test.model_name.sample(5)

17907         M_KLASSE
6148            ACCORD
14622            TEANA
15524    GL_KLASSE_AMG
23278        CARAVELLE
Name: model_name, dtype: object

Наименование модели. Принимается к рассмотрению.
### name

In [24]:
df_test.name.sample(5)

31688     500 5.0 AT (320 л.с.)
12014    2.0 CVT (141 л.с.) 4WD
12565    2.0 CVT (141 л.с.) 4WD
21361        2.0 CVT (148 л.с.)
25086        1.8 AMT (152 л.с.)
Name: name, dtype: object

Колонка содержит расширение названия модели, указывающее на обьем двигателя и тип привода. Поскольку эта информация уже получена из других колонок, эта в рассмотрнеие не принимается

### numberOfDoors

In [25]:
df_test.numberOfDoors.unique()

array([5, 4, 2, 3, 0], dtype=int64)

### parsing_unixtime
Колонка судя по названию содержит время сбора информации в валидационный набор данных. Непосредственно для обученяи информация не нужна. Но время сбора поможет внести поправку на изменение коньюктуры(курс валюты и т.д.). 
Посмотрим на диапазон `parsing_unixtime`:

In [26]:
date.fromtimestamp(df_test.parsing_unixtime.min()),date.fromtimestamp(df_test.parsing_unixtime.max())

(datetime.date(2020, 10, 19), datetime.date(2020, 10, 26))

Как видим парсинг произведен с 19 по 26 октября 20-го года
### productionDate

In [27]:
df_test.productionDate.unique()

array([2014, 2017, 2012, 2011, 2019, 2018, 2010, 2020, 2016, 2013, 2006,
       2007, 2015, 2005, 2008, 2009, 1997, 2004, 2002, 1987, 2003, 2001,
       1976, 2000, 1998, 1995, 1999, 1993, 1939, 1996, 1984, 1990, 1991,
       1992, 1989, 1982, 1985, 1994, 1938, 1981, 1988, 1983, 1980, 1986,
       1978, 1970, 1979, 1977, 1972, 1975, 1969, 1950, 1953, 1949, 1937,
       1959, 1968, 1936, 1904, 1974, 1967, 1961, 1960, 1965, 1963, 1957,
       1952, 1973, 1948], dtype=int64)

Год производства автомобиля ( наверное наряду с моделью автомобиля самы значимый признак). Принмается в рассмотрнеие.,
### sell_id

In [28]:
df_test.sell_id.unique()

array([1100575026, 1100549428, 1100658222, ..., 1101364889, 1101362518,
       1101256068], dtype=int64)

Колонка содержит ***id*** обьявления. Как непосредственный признак не интересен, в рассмотрение моделью не принимается. Но если предположить, что ***id*** отражает хронологический порядок появления обьявления, то, например, можно попробовать при "тонкой настройке" модели придать более новым обьявлениям больший вес при обучении. Оставим себе такую опцию, потому принимаем в рассмотрение.
### super_gen

In [29]:
df_test.super_gen[0]

'{"id":"10373605","displacement":1197,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL","transmission":"ROBOT","power":105,"power_kvt":77,"human_name":"1.2 AMT (105 л.с.)","acceleration":10.5,"clearance_min":155,"fuel_rate":5}'

Колонка содержит ***id*** и параметры автомобиля, которые либо являются по сути атрибутами модели (например у всех автомобилей одной модели одной модификации одинаковый клиренс) либо уже известны нам из других колонок (например тип трансмиссии). В рассмотрнеи не принимается.
### vehicleConfiguration

In [30]:
df_test.vehicleConfiguration.sample(5)

608      HATCHBACK_5_DOORS AUTOMATIC 1.6
28406     ALLROAD_5_DOORS MECHANICAL 1.6
33126      ALLROAD_5_DOORS AUTOMATIC 2.5
105        ALLROAD_5_DOORS AUTOMATIC 1.6
30783      ALLROAD_5_DOORS AUTOMATIC 3.6
Name: vehicleConfiguration, dtype: object

Содержит информацию о типе кузова, типе трансмиссии обьеме двигателя количестве дверей. Эта информация доступна в других колонках, поэтому `vehicleConfiguration` в рассмотрение не принимется 
### vehicleTransmission

In [31]:
df_test.vehicleTransmission.unique()

array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
      dtype=object)

Колонка содержит в тестовом виде тип трансмиссии. Принимается в рассмотрение.
### vendor

In [32]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [33]:
df_test.groupby('vendor').brand.unique()

vendor
EUROPEAN      [SKODA, AUDI, VOLVO, BMW, MERCEDES, VOLKSWAGEN]
JAPANESE    [HONDA, NISSAN, INFINITI, TOYOTA, LEXUS, MITSU...
Name: brand, dtype: object

Как видим колонка содержит обобщающий признак - принадлежность марки к европейским либо японским. Как подсказывает опыт автовладельца динамика падения цены от срока владения у этих двух групп различается (японцы обычно дешевеют медленнее, особенно Toyota). Посчитаем признак полезным, примем в рассмотрение моделью.
### Владельцы

In [34]:
df_test.Владельцы.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

Колонка содержит в текстовом виде три категории количества владельцев авто. Примем в рассмотрение
### Владение <=================

In [35]:
df_test.Владение.unique()[:20]

array([nan, '3 года и 2 месяца', '11 месяцев', '4 года и 1 месяц',
       '2 года и 4 месяца', '1 месяц', '3 года и 10 месяцев',
       '2 года и 6 месяцев', '6 лет и 4 месяца', '2 месяца', '4 месяца',
       '1 год и 10 месяцев', '2 года и 7 месяцев', '7 лет и 11 месяцев',
       '7 лет и 5 месяцев', '1 год и 11 месяцев', '2 года и 1 месяц',
       '7 лет', '4 года и 2 месяца', '2 года и 8 месяцев'], dtype=object)

Колонка содержит время прошедшее с момента выпуска авто. Очевидно линейно зависит о даты выпуска автомобиля - исключается из рассмотрения. **<=================**
### ПТС

In [36]:
df_test.ПТС.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

Колонка содержит (если значение не nan) категорию ПТС. Принимается в рассмотрение.
### Привод

In [37]:
df_test.Привод.unique()

array(['передний', 'полный', 'задний'], dtype=object)

Колонка содержит категорию привода. Принимается в расмотрние.

### Руль

In [38]:
df_test.Руль.unique()

array(['Левый', 'Правый'], dtype=object)

Колонка содержит категрию право- или левосторонности управления . В рассмотрение.
### Колонки подлежащие рассмотрению моделью
Итого из валидационного набора данных, содержащихся в `df_test`,  для обработки данных и последующего построения модели будем спользовать информацию из следующих колонок:

In [39]:
data_columns = data_columns = ['bodyType', 'brand', 'car_url', 'color', 'description', 
                    'engineDisplacement', 'enginePower', 'equipment_dict','fuelType', 
                    'mileage', 'modelDate', 'model_name', 'numberOfDoors', 'productionDate',
                    'sell_id', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение',
                    'ПТС', 'Привод', 'Руль']
# незабудем про целевую ппеременную
data_columns += ['offerprice']

# <a name="2"></a> Сбор и подгтовка данных
## <a name="2_0"></a> Парсинг данных (https://auto.ru/moskva/)
*вернуться к [Плану работы](#_)*

Данный блок производт парсинг обучающего набора данных 

Данные сбираются на сайте https://auto.ru/,  
из них формируется *pd.DataFrame* со следуюшими колонками:   
  
**bodytype** -   наименование типа кузова  
**brand** -  наименование марки  
**car_url** -  ссылка на обьявления о продаже  
**color** -  ссылки на обьявления о продаже  
**description** - текстовое описание продоваемого авто
**engineDisplacement** -  обьем двигателя  
**enginePower** -  мощность двигателя  
**equipment_dict** -  словарь с перечислением оснащения автомобиля.  
**fuelType** -  тип топлива  
**mileage** -  пробег авто  
**modelDate** -  год начала выпуска модели  
**model_name** -  наименование модели  
**numberOfDoors** -  количество дверей  
**productionDate** -  Год производства автомобиля  
**sell_id** -  содержит id обьявления   
**vehicleTransmission** -  тип трансмиссии  
**vendor** -  обобщающий признак: принадлежность марки к европейским либо японским маркам  
**Владельцы** -  количество владельцев авто 
**Владение** - срок владениея продавцем продаваемым автомобилем
**ПТС** -  Колонка содержит ('Оригинал', 'Дубликат') категорию ПТС  
**Привод** -   категория привода  
**Руль** -   категрию право- или левосторонности управления  
**offerprice** - цена продпажи (целевая переменная)

### Парсинг вспомогательных данных
Создаем и наполняем ***`marks_models `*** , получаем ***`marks_models_for_parsing`***  

***marks_models*** - словарь в которм:   
*ключ*:  обозначения марки на сайте auto.ru  
*значения*: списки всех моделей для каждой марки.  

***marks_models_for_parsing*** - содержит только марки, присутсвующие в валидационном наборе данных **test**  

затем сохраняем / читаем сохраненный ***marks_models_for_parsing***

In [42]:
'SKODA' in marks_for_parsing

True

In [ ]:
# 
marks_models_switch = 'данные уже есть'
# marks_models_switch = 'данных нет'

if marks_models_switch == 'данных нет':
    marks_models = get_marks_models()
    marks_models_for_parsing =  {k: v for k, v in marks_models.items() if k in marks_for_parsing}
    with open('./Project_7_data/marks_models_for_parsing.json', 'w') as f: 
        json.dump(marks_models_for_parsing, f)
elif marks_models_switch == 'данные уже есть':
    with open('./Project_7_data/marks_models_for_parsing.json') as f: 
        marks_models_for_parsing = json.loads(f.read())
else:
    raise Exception(
        "marks_models_switch должен иметь значения: 'данные уже есть' или 'данных нет' см. line 2")
    

### Создаем и заполняем `model_generation_year`
***model_generation_year*** - **pd.DataFrame**  в котором:  
        *full_name* - полное название марки и модели с указанием поколения;   
        *bodytype* - тип кузова поколения модели;  
        *generation_year* - год начала выпуска поколения  

Этот DataFrame нужен для преобразования полного названия модели в год начала выпуска модели

In [ ]:
# ПОЯСНЕНИЯ 
# model_generation_year_all =  get_model_generation_year(marks_models_for_parsing)
# model_generation_year_all.to_csv('./Project_5_data/model_generation_year.csv',index=False)
model_generation_year = pd.read_csv('./Project_7_data/model_generation_year.csv')

## Парсинг набора данных для обучения
Получение данных по маркам, присутсвующим в валидационном наборе данных

In [ ]:
# 
parsing_switch = 'данные уже есть'
# parsing_switch = 'данных нет'

if parsing_switch == 'данных нет':
    train = pd.DataFrame(columns=data_columns)
    for mark in marks_models_for_parsing: 
        for model in marks_models_for_parsing[mark]:      
            model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/' + '?output_type=table'
            # просмотр последовательно всех страниц текущей модели текущей марки
            for pages_num in range(1,10): 
                if pages_num==1: page_url = model_url
                else:            page_url = model_url + '&page=' +  str(pages_num)  
                # получем html страницы
                driver = webdriver.Chrome(executable_path)
                driver.maximize_window()
                driver.get(page_url)
                page_html = driver.execute_script("return document.body.innerHTML;")
                # создаем обьект bs4.BeautifulSoup из html страницы
                page_bs = BeautifulSoup(page_html, 'html.parser') 
                # список html-ек карточек на странице                                               
                tickets_on_page = page_bs.find_all('a', class_='ListingItemTitle__link') 
                # выход по исчерпанию страниц текущей модели текущей марки
                if not tickets_on_page: 
                    driver.quit()
                    break            
                # обработка карточек на странице
                for ticket in tickets_on_page:
                    # получаем url карточки текущего обьявления
                    ticket_url = ticket.get('href')
                    # извлекаем признаки и заполняем строку train-а
                    train.loc[len(train)] =get_features_from_ticket(ticket_url,driver)
                    time.sleep(0.5) 
                time.sleep(0.5) 
                driver.quit()
    train.to_csv('./Project_7_data/train_mod_2.csv',index=False)
    
elif parsing_switch == 'данные уже есть':
    train = pd.read_csv('./Project_7_data/train_mod_2.csv')
else:
    raise Exception(
        "parsing_switch должен иметь значения: 'данные уже есть' или 'данных нет' см. line 2")

## <a name="2_1"></a> Привлечение сторонних данных
вернуться к [Плану работы](#_)

In [ ]:
df= pd.read_csv('./Project_7_data/' + 'auto_ru_2020_09_09.csv',low_memory=False)
df_full= pd.read_csv('./Preproject_7_data/' + 'all_auto_ru_09_09_2020.csv',low_memory=False)

## <a name="2_2"></a> Предобработка полученых данных
вернуться к [Плану работы](#_)
### <a name="2_2_0">Дубли и пропуски
Сначала посмотрим, есть ли вообще дубликаты в данных и, если есть, скольько их.

Собственные данные

In [ ]:
print(f'В собственных данных {train.duplicated().sum()} дупликатов')
print(f'В сторонних данных {df_full.duplicated(subset=test_0909_uni_0909[:-1]).sum()} дупликатов')

Значительоне количество в обоих датасетах, удаляем.

In [ ]:
df_full.drop_duplicates(subset=test_0909_uni_0909[:-1],keep='last',inplace=True) 

### <a name="2_2_1">Приведение к единому виду
Теперь приведем к единому виду

In [ ]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)

# <a name="3"></a>EDA полученных данных
## <a name="3_0"></a>EDA собсвенного набора
вернуться к [Плану работы](#_)

## <a name="3_1">EDA сторонних данных
вернуться к [Плану работы](#_)

## <a name="3_2">Сравнение распределений в наборах данных
вернуться к [Плану работы](#_)

## <a name="3_3"> Выводы, предварительный выбор модели, варианты обработки данных
вернуться к [Плану работы](#_)

# <a name="4">Создание, настройка модели, участие в соревновании
вернуться к [Плану работы](#_)

# <a name="5">Ансамблирование
вернуться к [Плану работы](#_)

# ====================== TEST & OTHER ==================

In [ ]:
# # альтернативные пути для kaggle и локальный (НЕ ЗАБЫВАТЬ МЕНЯТЬ!)
# path = './Project_7_data/'
# # path = '/kaggle/input/.....'

# # фиксация randomstate
# RANDOM_SEED = 42

# executable_path = 'C:/Users/GANSOR-PC/chromium/chromedriver.exe'
# marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
#        'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

# data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
#        'enginePower', 'equipment_dict', 'fuelType', 'mileage', 'modelDate',
#        'model_name', 'numberOfDoors', 'productionDate', 'sell_id', 'vehicleTransmission',
#        'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'offerprice'] 

# # data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
# #                       'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
# #                       'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
# #                       'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']